In [1]:
from __future__ import division
from __future__ import print_function

import random
import os
from collections import defaultdict
import time
from statistics import median
import json
from tracemalloc import start

import numpy as np
import tensorflow as tf
from gcn.utils import *
from gcn.models import GCN_DEEP_DIVER
from graph_methods import *

/home/abi/miniconda3/envs/tf_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/abi/miniconda3/envs/tf_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/abi/miniconda3/envs/tf_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/abi/miniconda3/envs/tf_env/lib/python3.6/site-

In [24]:
def getBestMDS(adj, predictions):
    start = time.time()
    g = nx.from_numpy_matrix(adj)

    bestSolution = list(g)
    solution_sizes = []

    runtimes = []

    for prediction in predictions.transpose():
        tmpTime = time.time()
        potentialSolution = buildMDS(g, prediction)
        bestSolution = potentialSolution if len(potentialSolution) < len(bestSolution) else bestSolution
        solution_sizes.append(len(potentialSolution))
        runtimes.append(time.time() - tmpTime)
    
    return bestSolution, (time.time()-start), solution_sizes, mean(runtimes)

def buildMDS(g, prediction):
    test = time.time()
    sortedNodes = sorted(zip(list(g), prediction), key=lambda x: x[1], reverse=True)
    nodeOrder = [x[0] for x in sortedNodes]
    print(f"TIME TO SORT NODES: {time.time() - test}")

    # Build minimum dominating set using binary search
    test = time.time()
    min = 0
    max = len(nodeOrder) - 1
    while min < max:
        mid = (max + min) // 2
        currSolution = nodeOrder[:mid+1]

        if nx.algorithms.dominating.is_dominating_set(g, currSolution):
            max = mid
        else:
            min = mid + 1

    currSolution = nodeOrder[:min+1]
    assert(min == max)
    assert(nx.algorithms.dominating.is_dominating_set(g,currSolution))
    print(f"TIME TO COMPLETE BIN SEARCH: {time.time()-test}")

    # Prune the dominating set
    test = time.time()
    print(f"Nodes to prune: {len(currSolution)}")

    for i in range(len(currSolution) - 1, -1, -1):
        # newSolution = currSolution[:i] + currSolution[i+1:]
        currSolution[i], currSolution[-1] = currSolution[-1], currSolution[i]
        tmp = currSolution.pop()

        if not nx.algorithms.dominating.is_dominating_set(g, currSolution):
            currSolution.append(tmp)
                
    assert(nx.algorithms.dominating.is_dominating_set(g, currSolution))

    print(f"TIME TO PRUNE NODES: {time.time() - test}")
    return sorted(currSolution)






def greedySolution(adj):
    start_time = time.time()

    g = nx.from_numpy_matrix(adj)

    dominatingSet = []
    dominatedNodes = set()

    while not nx.algorithms.dominating.is_dominating_set(g, dominatingSet):
        currNodes = []
        max_white_nodes = 0
        for v in g:
            if v in dominatingSet:
                continue

            white_count = 0
            for u in g.neighbors(v):
                if u not in dominatedNodes:
                    white_count += 1
            if v not in dominatedNodes:
                white_count += 1
            
            if white_count == max_white_nodes:
                currNodes.append(v)
            elif white_count > max_white_nodes:
                currNodes = [v]
                max_white_nodes = white_count
        
        dominatingSet.extend(currNodes)
        for node in currNodes:
            dominatedNodes.add(node)
            for adj_node in g.neighbors(node):
                dominatedNodes.add(adj_node)
    
    greedySize = len(dominatingSet)

    print(f"NODES BEING PRUNED BY GREEDY: {len(dominatingSet)}")
    test = time.time()
    # Prune the dominating set
    while True:
        remove = False

        for i in range(len(dominatingSet) - 1, -1, -1):
            newSolution = dominatingSet[:i] + dominatingSet[i+1:]

            if nx.algorithms.dominating.is_dominating_set(g, newSolution):
                remove = True
                dominatingSet = newSolution
                break

        if not remove:
            break
    print(f"TIME FOR GREEDY PRUNING: {time.time() - test}")
    
    assert(nx.algorithms.dominating.is_dominating_set(g, dominatingSet))

    prunedGreedySize = len(dominatingSet)

    return greedySize, prunedGreedySize, time.time() - start_time

In [3]:
RUN_NAME = "FINAL_RUN"
N_bd = 32

# Settings
flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_string('model', 'gcn_cheby', 'Model string.')  # 'gcn', 'gcn_cheby', 'dense'
flags.DEFINE_float('learning_rate', 0.001, 'Initial learning rate.')
flags.DEFINE_integer('epochs', 250, 'Number of epochs to train.')
flags.DEFINE_integer('hidden1', 32, 'Number of units in hidden layer 1.')
flags.DEFINE_integer('diver_num', 32, 'Number of outputs.')
flags.DEFINE_float('dropout', 0, 'Dropout rate (1 - keep probaNUmbility).')
flags.DEFINE_float('weight_decay', 5e-4, 'Weight for L2 loss on embedding matrix.')
flags.DEFINE_integer('early_stopping', 1000, 'Tolerance for early stopping (# of epochs).')
flags.DEFINE_integer('max_degree', 1, 'Maximum Chebyshev polynomial degree.')
flags.DEFINE_integer('num_layer', 20, 'number of layers.')

# Some preprocessing
num_supports = 1 + FLAGS.max_degree
model_func = GCN_DEEP_DIVER

# Define placeholders
placeholders = {
    'support': [tf.sparse_placeholder(tf.float32) for _ in range(num_supports)],
    'features': tf.sparse_placeholder(tf.float32, shape=(None, N_bd)), # featureless: #points
    'labels': tf.placeholder(tf.float32, shape=(None, 2)), # 0: not linked, 1:linked
    'labels_mask': tf.placeholder(tf.int32),
    'dropout': tf.placeholder_with_default(0., shape=()),
    'num_features_nonzero': tf.placeholder(tf.int32)  # helper variable for sparse dropout
}

# Create model
model = model_func(placeholders, input_dim=N_bd, logging=True)

# use gpu 0
os.environ['CUDA_VISIBLE_DEVICES']=str(0)

# Initialize session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

# Init variables
saver=tf.train.Saver(max_to_keep=1000)
sess.run(tf.global_variables_initializer())

ckpt=tf.train.get_checkpoint_state(f"{RUN_NAME}")
if ckpt:
    print('loaded '+ckpt.model_checkpoint_path)
    saver.restore(sess,ckpt.model_checkpoint_path)


def get_real_graphs():
    GRAPH_PATH = "./REDDIT-MULTI-5K"

    def get_node_map():
        node_map = [None]

        with open(os.path.join(GRAPH_PATH, "REDDIT-MULTI-5K.graph_idx"), "r") as f:
            for node_id, graph_id in enumerate(f, 1):
                node_map.append(int(graph_id))

        return node_map

    def get_edge_lists(node_map = None):
        if not node_map:
            node_map = get_node_map()

        edge_lists = defaultdict(list)

        with open(os.path.join(GRAPH_PATH, "REDDIT-MULTI-5K.edges"), "r") as f:
            for line in f:
                n1, n2 = line.strip('\n').split(',')
                edge_lists[node_map[int(n1)]].append(f"{n1} {n2}")

        return edge_lists

    def create_graphs(edge_lists = None):

        if not edge_lists:
            edge_lists = get_edge_lists()

        graphs = {}

        for graph_id in edge_lists:
            graphs[graph_id] = nx.parse_edgelist(edge_lists[graph_id], nodetype=int)

        return graphs

    return create_graphs()

# Define model evaluation function
def testingEvaluataion(features, support, placeholders):
    t_test = time.time()
    feed_dict_val = construct_feed_dict4pred(features, support, placeholders)
    outs_val = sess.run([model.outputs_softmax], feed_dict=feed_dict_val)
    return (time.time() - t_test), outs_val[0]

# tuples stored as (gamma, greedy, GCN)
testing_analysis = {}

graphs = get_real_graphs()
sorted_graphs = sorted(graphs.values(), key=lambda x: len(x))

loaded FINAL_RUN/model.ckpt
INFO:tensorflow:Restoring parameters from FINAL_RUN/model.ckpt


In [25]:
graph = sorted_graphs[3248]
adj = nx.adjacency_matrix(graph).todense()
updatedGreedy, prunedGreedy, newGreedyTime = greedySolution(adj)

NODES BEING PRUNED BY GREEDY: 196
TIME FOR GREEDY PRUNING: 0.32535552978515625


In [26]:
nn = adj.shape[0]
features = np.ones([nn, N_bd])
features = sp.lil_matrix(features)
features = preprocess_features(features)
support = simple_polynomials(adj, FLAGS.max_degree)
tmpRuntime, outs = testingEvaluataion(features, support, placeholders)

In [27]:
sol, totalTime, solution_sizes, avgTime = getBestMDS(adj, outs)

TIME TO SORT NODES: 0.0001895427703857422
TIME TO COMPLETE BIN SEARCH: 0.004936695098876953
Nodes to prune: 424
TIME TO PRUNE NODES: 0.19601869583129883
TIME TO SORT NODES: 0.00021839141845703125
TIME TO COMPLETE BIN SEARCH: 0.005022764205932617
Nodes to prune: 410
TIME TO PRUNE NODES: 0.1299889087677002
TIME TO SORT NODES: 0.00020241737365722656
TIME TO COMPLETE BIN SEARCH: 0.0057146549224853516
Nodes to prune: 467
TIME TO PRUNE NODES: 0.20822596549987793
TIME TO SORT NODES: 0.0002181529998779297
TIME TO COMPLETE BIN SEARCH: 0.005609273910522461
Nodes to prune: 489
TIME TO PRUNE NODES: 0.2124025821685791
TIME TO SORT NODES: 0.0002167224884033203
TIME TO COMPLETE BIN SEARCH: 0.0054509639739990234
Nodes to prune: 458
TIME TO PRUNE NODES: 0.21439909934997559
TIME TO SORT NODES: 0.0002167224884033203
TIME TO COMPLETE BIN SEARCH: 0.0038404464721679688
Nodes to prune: 275
TIME TO PRUNE NODES: 0.06996917724609375
TIME TO SORT NODES: 0.00021386146545410156
TIME TO COMPLETE BIN SEARCH: 0.00559